# Primeira e Segunda Parte

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import random
import pandas as pd
import numpy as np
from scipy import stats
import networkx as nx
from scipy.stats import poisson
from scipy.stats import nbinom
from timeit import default_timer as timer
from plotly.subplots import make_subplots

In [ ]:
r = 0.5
mu = 6

p = r/(r+mu)
print(p)

In [ ]:
N = [100,1000,3000,5000,8000, 10000]

In [ ]:
dist = []
for i in range(len(N)):

  nb = nbinom.rvs(r, p, size=N[i])

  while sum(nb) % 2 != 0:
    nb = nbinom.rvs(r, p, size=N[i])

  dist.append(nb)

print(dist[0])


In [ ]:
teste = nbinom.rvs(r, p, size=50)

while sum(teste) % 2 != 0:
  teste = nbinom.rvs(r, p, size=50)

G_teste = nx.configuration_model(teste)
G_teste.remove_edges_from(nx.selfloop_edges(G_teste))

G_100 = nx.configuration_model(dist[0])
G_100.remove_edges_from(nx.selfloop_edges(G_100))

G_1000 = nx.configuration_model(dist[1])
G_1000.remove_edges_from(nx.selfloop_edges(G_1000))

G_3000 = nx.configuration_model(dist[2])
G_3000.remove_edges_from(nx.selfloop_edges(G_3000))

G_5000 = nx.configuration_model(dist[3])
G_5000.remove_edges_from(nx.selfloop_edges(G_5000))

G_8000 = nx.configuration_model(dist[4])
G_8000.remove_edges_from(nx.selfloop_edges(G_8000))

G_10000 = nx.configuration_model(dist[5])
G_10000.remove_edges_from(nx.selfloop_edges(G_10000))

In [ ]:
pos = nx.circular_layout(G_teste)
plt.figure(figsize=(10,10))
plt.axis("off")
nx.draw_networkx_nodes(G_teste,pos, node_size=300, node_color="black")
nx.draw_networkx_edges(G_teste,pos, alpha=0.500)
nx.draw_networkx_labels(G_teste, pos, font_color="white")
plt.show()

In [ ]:
def gillespie_simulation(G, states, beta, gamma, max_time,N):
    t = 0
    time = []
    susceptible = []
    infected = []
    recovered = []
    rho_i_t = []
    aux = np.inf


    while t < max_time:  # Enquanto tempo não exceder o máximo
        rates = []
        events = []

        for node in range(N):
            if states[node] == 1:
              for neighbor in G.neighbors(node):
                  if states[neighbor] == 0:
                      rates.append(beta)
                      events.append(('infection', node, neighbor))
              rates.append(gamma)
              events.append(('recovery', node))

        if not rates:
            break

        rate_sum = sum(rates)
        delta_t = np.random.exponential(1 / rate_sum)
        t += delta_t

        chosen_event = np.random.choice(len(events), p=np.array(rates) / rate_sum)
        event_type, node, *args = events[chosen_event]

        if event_type == 'infection':
            states[args[0]] = 1  # Suscetível -> Infectado
        elif event_type == 'recovery':
            states[node] = 2  # Infectado -> Recuperado

        time.append(t)
        susceptible.append(np.sum(states == 0))
        infected.append(np.sum(states == 1))
        recovered.append(np.sum(states == 2))

        if abs(aux - np.sum(states == 2)/G.number_of_nodes()) < 1e-3:
          rho_i_t.append(np.sum(states == 2)/G.number_of_nodes())
        aux = np.sum(states == 2)/G.number_of_nodes()


    return time, susceptible, infected, recovered, rho_i_t

In [ ]:
beta = 0
gamma = 1
initial_infected = 10
max_time = 200

In [ ]:
def plot_infection(N,G):
  states = np.zeros(N, dtype=int)

  # Definir nós infectados iniciais
  initial_infected_nodes = np.random.choice(N, initial_infected, replace=False)
  states[initial_infected_nodes] = 1


  start = timer()

  time, susceptible, infected, recovered, rho_t = gillespie_simulation(G, states, beta, gamma, max_time,N)

  end = timer()

  t = end - start

  # Plotar os resultados usando Plotly
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=time, y=susceptible, mode='lines', name='Susceptible', line=dict(color='blue')))
  fig.add_trace(go.Scatter(x=time, y=infected, mode='lines', name='Infected', line=dict(color='red')))
  fig.add_trace(go.Scatter(x=time, y=recovered, mode='lines', name='Recovered', line=dict(color='green')))

  fig.update_layout(title="Simulação de Epidemia com Algoritmo de Gillespie",
                    xaxis_title="Tempo",
                    yaxis_title="Número de Indivíduos",
                    legend_title="Estado",
                    width=900, height=600)

  fig.show()
  return fig,t

## N = 100

In [ ]:
fig1, t0 = plot_infection(N[0],G_100)

In [ ]:
print('Demorou',t0, 'segundos')

## N = 1000

In [ ]:
fig2,t1 = plot_infection(N[1],G_1000)

In [ ]:
print('Demorou',t1, 'segundos')

## N = 3000

In [ ]:
fig3,t2 = plot_infection(N[2],G_3000)

In [ ]:
print('Demorou',t2, 'segundos')

## N = 5000

In [ ]:
fig4,t3 = plot_infection(N[3],G_5000)

In [ ]:
print('Demorou',t3, 'segundos')

## N = 8000

In [ ]:
fig5,t4 = plot_infection(N[4],G_8000)

In [ ]:
print('Demorou',t4, 'segundos')

## N = 10000

In [ ]:
fig6,t5 = plot_infection(N[5],G_10000)

In [ ]:
print('Demorou',t5, 'segundos')

In [ ]:
def plot_sub(plots):
  fig = make_subplots(rows=3, cols=2, subplot_titles=("N = 100", "N = 1000", "N = 3000", "N = 5000", "N = 8000","N = 10000"),
                    specs=[[{"type": "scatter"}, {"type": "scatter"}], [{"type": "scatter"},
                           {"type": "scatter"}], [{"type": "scatter"}, {"type": "scatter"}]])

  for i, subplot_fig in enumerate(plots, start=1):
      show_legend = True if i == 1 else False
      for trace in subplot_fig.data:
          trace.showlegend = show_legend
          fig.add_trace(trace, row=(i - 1) // 2 + 1, col=(i - 1) % 2 + 1)

  # Ajustando o layout
  fig.update_layout(height=800, width=1000, title_text="Simulations of epidemics using the Gillespie algorithm and the configuration model")

  # Exibindo o gráfico
  fig.show()

  return

## N = 100000 (Demorou mais que 9 horas e nem terminou...)

## Plotando os tempos

In [ ]:
import plotly.express as px
from scipy.optimize import curve_fit

In [ ]:
def plot_time(Ns,tempos):
  coefficients = np.polyfit(Ns, tempos, 2) #Complexidade O(n²) do algoritmo de Gillespie
  polynomial = np.poly1d(coefficients)

  print(coefficients)

  x = np.linspace(min(Ns), max(Ns), 500)
  y = polynomial(x)

  plt.scatter(Ns, tempos, label='Time(in seconds)', color='red')
  plt.plot(x,y, label='Polynomial adjust', color='blue')
  plt.xlabel("Size of the network")
  plt.ylabel("Seconds")
  plt.legend()
  plt.show()

  return x,y

In [ ]:
tempos = np.array([t0,t1,t2,t3,t4,t5])
Ns = np.array(N)
x,y = plot_time(Ns,tempos)

# Terceira Parte

In [ ]:
print(N)
print(mu)

In [ ]:
GR_100 = nx.random_regular_graph(mu,N[0])
GR_1000 = nx.random_regular_graph(mu,N[1])
GR_3000 = nx.random_regular_graph(mu,N[2])
GR_5000 = nx.random_regular_graph(mu,N[3])
GR_8000 = nx.random_regular_graph(mu,N[4])
GR_10000 = nx.random_regular_graph(mu,N[5])

In [ ]:
pos = nx.fruchterman_reingold_layout(GR_100)
plt.figure(figsize=(15,10))
plt.axis("off")
nx.draw_networkx_nodes(GR_100, pos, node_size=300, node_color="black")
nx.draw_networkx_edges(GR_100, pos, alpha=0.500)
nx.draw_networkx_labels(GR_100, pos, font_color="white")
plt.show()

In [ ]:
figr1,tr0 = plot_infection(N[0],GR_100)

In [ ]:
print('Demorou',tr0, 'segundos')

In [ ]:
figr2,tr1 = plot_infection(N[1],GR_1000)

In [ ]:
print('Demorou',tr1, 'segundos')

In [ ]:
figr3,tr2 = plot_infection(N[2],GR_3000)

In [ ]:
print('Demorou',tr2, 'segundos')

In [ ]:
figr4,tr3 = plot_infection(N[3],GR_5000)

In [ ]:
print('Demorou',tr3, 'segundos')

In [ ]:
figr5,tr4 = plot_infection(N[4],GR_8000)

In [ ]:
print('Demorou',tr4, 'segundos')

In [ ]:
figr6,tr5 = plot_infection(N[5],GR_10000)

In [ ]:
print('Demorou',tr5, 'segundos')

In [ ]:
plot_sub([figr1,figr2,figr3,figr4,figr5,figr6])

In [ ]:
tr = np.array([tr0,tr1,tr2,tr3,tr4,tr5])
xr,yr = plot_time(Ns,tr)

## Comparação random regular vs configuration model

In [ ]:
def plot_sub_comp1(plots):
  fig = make_subplots(rows=3, cols=2, subplot_titles=("N = 100 configuration", "N = 100 random regular", "N = 1000 configuration", "N = 1000 random regular", "N = 3000 configuration","N = 3000 random regular"),
                    specs=[[{"type": "scatter"}, {"type": "scatter"}], [{"type": "scatter"},
                           {"type": "scatter"}], [{"type": "scatter"}, {"type": "scatter"}]])

  for i, subplot_fig in enumerate(plots, start=1):
      show_legend = True if i == 1 else False
      for trace in subplot_fig.data:
          trace.showlegend = show_legend
          fig.add_trace(trace, row=(i - 1) // 2 + 1, col=(i - 1) % 2 + 1)

  # Ajustando o layout
  fig.update_layout(height=800, width=1000, title_text="Simulations of epidemics using the Gillespie algorithm, the configuration model and random regular networks")

  # Exibindo o gráfico
  fig.show()

  return

In [ ]:
def plot_sub_comp2(plots):
  fig = make_subplots(rows=3, cols=2, subplot_titles=("N = 5000 configuration", "N = 50000 random regular", "N = 8000 configuration", "N = 8000 random regular", "N = 10000 configuration","N = 10000 random regular"),
                    specs=[[{"type": "scatter"}, {"type": "scatter"}], [{"type": "scatter"},
                           {"type": "scatter"}], [{"type": "scatter"}, {"type": "scatter"}]])

  for i, subplot_fig in enumerate(plots, start=1):
      show_legend = True if i == 1 else False
      for trace in subplot_fig.data:
          trace.showlegend = show_legend
          fig.add_trace(trace, row=(i - 1) // 2 + 1, col=(i - 1) % 2 + 1)

  # Ajustando o layout
  fig.update_layout(height=800, width=1000, title_text="Simulations of epidemics using the Gillespie algorithm, the configuration model and random regular networks")

  # Exibindo o gráfico
  fig.show()

  return

In [ ]:
plot_sub_comp1([fig1,figr1,fig2,figr2,fig3,figr3])

In [ ]:
plot_sub_comp2([fig4,figr4,fig5,figr5,fig6,figr6])

In [ ]:
plt.plot(x,y, label='Configuration model adjust', color='C0')
plt.scatter(Ns, tempos, label='Time configuration', color='C0')
plt.plot(xr,yr, label='Random regular adjust', color='C3')
plt.scatter(Ns, tr, label='Time random regular', color='C3')
plt.xlabel("Size of the network")
plt.ylabel("Seconds")

# Remove a borda superior (spine top)
ax = plt.gca()  # pega os eixos atuais
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)  # direita

plt.legend()
plt.show()

### O algoritmo demora mais em random regular networks. Pode ser porque todos os nós estão conectados, isto é, só há uma componente conexa e com isso, o algoritmo analisa muito mais nós e interações.